In [1]:
import numpy as np
from collections import Counter, defaultdict

vocab = 'anpa ante awen esun insa jaki jelo kala kama kasi kili kule kute lape laso lawa lete lili lipu loje luka lupa mama mani meli mije moku moli musi mute nasa nena nimi noka olin open pali pana pini pipi poka poki pona sama seli selo seme sewi sike sina sona suli suno supa suwi taso tawa telo toki tomo unpa walo waso wawa weka wile'.split()
print(len(vocab))
chars = list(set([char for word in vocab for char in word]))
chars.sort()
print(len(chars))

def get_hint(ans, cand):
    show = [2, 2, 2, 2] # 0: green, correct(C), 1: yellow, wrong(W), 2: no (X)
    
    # 2 -> 0
    for i in range(4):
        if ans[i] == cand[i]:
            show[i] = 0 # correct
    
    # 2 -> 1
    # candidate characters for yellow character
    rest = list(set([cand[i] for i in range(4) if show[i] == 2]))
    for r in rest: 
        if any(r == a for i, a in enumerate(ans) if show[i] in {1, 2}):
            for i in range(4): # search index to yellow
                if cand[i] == r and show[i] == 2:
                    show[i] = 1
                    break
    hint = ''.join(['0', '1', '2'][n] for n in show)
    return hint

print(get_hint('ante', 'wawa')) # 正解 ante に対して，候補 wawa を入力した結果 -> XWXX
print(get_hint('anpa', 'sina')) # 正解 anpa に対して，候補 sina を入力した結果 -> XXWC
print(get_hint('sina', 'anpa')) # 正解 sina に対して，候補 anpa を入力した結果 -> XWXC
print(get_hint('nena', 'anpa')) # 正解 nena に対して，候補 anpa を入力した結果 -> XWXC
print(get_hint('nasa', 'anpa')) # 正解 nasa に対して，候補 anpa を入力した結果 -> WWXC

def filter_words(words, hint, cand):
    words = [x for x in words if x != cand]
    
    # Wの定義を，厳密に「ここではなく他のところにある」にしてjを書き換え
    yellow_chars = ''
    for i in range(4):
        if hint[i] in 'CW':
            yellow_chars += cand[i] # 少なからずあたっている
    for i in range(4):
        if (cand[i] in yellow_chars) and (hint[i] == 'X'):
            hint = hint[:i] + 'W' + hint[i+1:]
    
    for i in range(4):
        if hint[i] == 'C':
            words = [word for word in words if word[i] == cand[i]]
        elif hint[i] == 'W':
            words = [word for word in words if cand[i] in word]
            words = [word for word in words if word[i] != cand[i]]
        elif hint[i] == 'X':
            words = [word for word in words if cand[i] not in word]
            
    return words

print(filter_words(vocab, 'XWXX', 'wawa')) # 初手wawaに対する回答がXWXXだったとき，次に候補となる単語のリスト
print(filter_words(['kama', 'kasi'], 'CCXX', 'kama'))

def split_cand(words, cand):
    dct = defaultdict(list)
    for word in words:
        dct[get_hint(word, cand)].append(word)
    return dict(dct)

def count_cand(words, cand):
    dct = split_cand(words, cand)
    dct = {key: len(value) for key, value in dct.items()}
    return dct

def calc_ent(words, cand):
    dct = count_cand(words, cand)
    ent = 0
    denom = sum(dct.values())
    for i in dct.values():
        p = i / len(words)
        ent -= p * np.log2(p)
    return ent

66
14
2122
2210
2120
2120
1120
['ante']
['kasi']


In [2]:
# 1語だけでのエントロピーが大きくなるような語を列挙する
# この中から，なるべく多く3手で全部網羅できるのをひとつひとつ探したらwaloでできてしまった
ents = [(word, calc_ent(vocab, word)) for word in vocab]
ents.sort(key = lambda x: -x[1])
ents[:20]

[('laso', 4.441508514179351),
 ('seli', 4.3505994232702605),
 ('selo', 4.345318884700614),
 ('sina', 4.263741945530095),
 ('lape', 4.219632176743091),
 ('pali', 4.21794961263142),
 ('sona', 4.209357788586154),
 ('sike', 4.20140083556324),
 ('meli', 4.164487015692192),
 ('kasi', 4.158572477426081),
 ('walo', 4.148402787392285),
 ('suli', 4.144784136520656),
 ('mani', 4.120924946057055),
 ('moli', 4.099779766428733),
 ('luka', 4.09145695284965),
 ('pona', 4.061153922546621),
 ('wile', 4.053092271400567),
 ('poka', 4.048839620832478),
 ('noka', 4.027827167076222),
 ('waso', 3.9943907110834886)]

In [3]:
# 'walo'の部分を書き換えると，3手でなるべく分けてくれる
# 入力することでエントロピーが最大になるような単語のみでの探索なので，もしかしたら他にも手があるかもしれない
for key, value in split_cand(vocab, 'walo').items():
    ents = [(word, calc_ent(value, word)) for word in vocab]
    ents.sort(key = lambda x: -x[1])
    best, ent = ents[0]
    dct = split_cand(value, best)
    print(key, best, dct)

2122 nasa {'1120': ['anpa'], '1122': ['ante'], '1200': ['insa'], '0220': ['nena'], '1210': ['sina'], '2210': ['supa'], '1220': ['unpa']}
1122 anpa {'0122': ['awen']}
2222 mani {'2212': ['esun'], '2222': ['kute'], '0221': ['mije'], '0220': ['musi'], '0222': ['mute'], '1210': ['nimi'], '2200': ['pini'], '2220': ['pipi'], '1222': ['seme'], '2221': ['sike']}
2022 musi {'2220': ['jaki'], '1222': ['kama'], '2200': ['kasi'], '0222': ['mama'], '0220': ['mani'], '2202': ['nasa'], '2222': ['pana'], '1212': ['sama']}
2200 taso {'2220': ['jelo'], '2210': ['selo'], '0220': ['telo']}
2002 anpa {'1220': ['kala'], '1212': ['pali']}
2202 sike {'2012': ['kili'], '2210': ['kule'], '2022': ['lili'], '2121': ['meli'], '0121': ['seli'], '0122': ['suli']}
2012 anpa {'1202': ['lape']}
2010 anpa {'1222': ['laso']}
1012 anpa {'1220': ['lawa']}
2212 anpa {'2222': ['lete'], '2202': ['lipu']}
2211 anpa {'2222': ['loje'], '2122': ['olin']}
2112 anpa {'2220': ['luka'], '2200': ['lupa']}
2221 kute {'1122': ['moku'], 